In [37]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from centris.centris_scraper import CentrisBienParser, AsyncCentrisScraper
from centris.db_models import PlexCentrisListingDB
from centris import Session
from sqlalchemy.sql import select

from pprint import pprint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Scraping 1 listing

In [21]:
plex_start_url="https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1"
url_bien_centris = "https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary"

In [22]:
centris_parser = CentrisBienParser(url_bien_centris)

In [32]:
scrape_date=datetime.now()
data = centris_parser.get_data(scrape_date)

In [30]:
pprint(data.model_dump())


{'adresse': "10627 - 10631, Rue D'Iberville, Montréal (Ahuntsic-Cartierville), "
            'Quartier Ahuntsic Est',
 'annee_construction': 1966,
 'centris_id': 19418151,
 'date_scrape': '2024-12-28',
 'description': None,
 'eval_municipale': 815000,
 'nombre_unites': 3,
 'prix': 980000,
 'quartier': 'Ahuntsic Cartierville',
 'revenus': 36000,
 'stationnement': 3,
 'style_batiment': None,
 'superficie_batiment': None,
 'superficie_commerce': None,
 'superficie_habitable': None,
 'superficie_terrain': 3401,
 'taxes': 5313,
 'title': 'Triplex à vendre',
 'unites': ['3 1/2', '5 1/2', '5 1/2'],
 'url': 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary',
 'utilisation': 'Résidentielle',
 'ville': 'Montreal'}


saving to DB

In [ ]:
db_entry = centris_parser.to_db_model(scrape_date)
with Session.begin() as session:
    session.add_all(
        [
            db_entry
        ]
    )
    session.commit()

In [ ]:
# Query the database for testing
stmt = select(PlexCentrisListingDB)
objects = session.scalars(stmt).all()

In [44]:
objects[0].__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x11fce7770>,
 'unites': '["3 1/2", "5 1/2", "5 1/2"]',
 'utilisation': 'Résidentielle',
 'eval_municipale': 815000,
 'nombre_unites': 3,
 'style_batiment': None,
 'date_scrape': '2024-12-28',
 'url': 'https://www.centris.ca/fr/triplex~a-vendre~montreal-ahuntsic-cartierville/19418151?view=Summary',
 'superficie_habitable': None,
 'adresse': "10627 - 10631, Rue D'Iberville, Montréal (Ahuntsic-Cartierville), Quartier Ahuntsic Est",
 'superficie_batiment': None,
 'ville': 'Montreal',
 'centris_id': 19418151,
 'superficie_commerce': None,
 'quartier': 'Ahuntsic Cartierville',
 'title': 'Triplex à vendre',
 'superficie_terrain': 3401,
 'prix': 980000,
 'revenus': 36000,
 'annee_construction': 1966,
 'stationnement': 3,
 'description': None,
 'taxes': 5313}

### Scraping batch of listing

In [ ]:
scraper = AsyncCentrisScraper(plex_start_url)
urls = await scraper.navigate_centris()

Critical error in navigation: Page.goto: net::ERR_HTTP_RESPONSE_CODE_FAILURE at https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1
Call log:
  - navigating to "https://www.centris.ca/fr/plex~a-vendre~montreal?view=Thumbnail&uc=1", waiting until "load"

